In [1]:
import pandas as pd
import numpy as np 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\maity\AppData\Roaming\nltk_data...


True

## Load the dataset as pandas

In [19]:
df_train_pos = pd.read_csv("twitter-datasets/train_pos.txt",sep="\t",names=['tweets'])
df_train_neg = pd.read_csv("twitter-datasets/train_neg.txt",sep="\t",names=['tweets'])
print(df_train_pos.shape)
print(df_train_neg.shape)

(97902, 1)
(99068, 1)


In [20]:
df_train_pos.head()

,tweets
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,<user> just put casper in a box ! looved the...
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...


## Remove tags and urls

In [21]:
def remove_tags(df):
    df_cleaned = df.copy()
    df_cleaned['tweets'] = df_cleaned['tweets'].apply(lambda tweet: re.sub(r'<.*?>', '', tweet).strip())
    return df_cleaned

In [22]:
remove_tags(df_train_pos)

,tweets
0,i dunno justin read my mention or not . only j...
1,"because your logic is so dumb , i won't even c..."
2,just put casper in a box ! looved the battle ...
3,thanks sir > > don't trip lil mama ... just ke...
4,visiting my brother tmr is the bestest birthda...
...,...
97897,hey gina what's up ?
97898,"sas 9.1 . 3 and 9.2 , east 5 , s-plus 8 , stat..."
97899,um gord ... i just read your profile . i'm not...
97900,i'm so excited for tomorrow ! look out for two...


In [26]:
# Example to show the use of strip()
s = "<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me "
# Add the strip() method to remove unecessary spaces in the tweet after removing tags and urls
tokens = word_tokenize(re.sub(r'<.*?>', '', s).strip())
stop_words = stopwords.words('english')
l = [token for token in tokens if token.lower() not in stop_words]
print(l)
ps = PorterStemmer()
l_after_stem = [ps.stem(token) for token in l]
print(l_after_stem)
wordnet_lemmatizer = WordNetLemmatizer()
l_after_lem = [wordnet_lemmatizer.lemmatize(token) for token in l]
print(l_after_lem)

['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'knows', ',', 'hope', 'follow']
['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'know', ',', 'hope', 'follow']
['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'know', ',', 'hope', 'follow']


## Removing stop words + stemming

In [24]:
def tokenize_and_preprocess(df, stop_words = False, stemming = False, lemmatization = False):
    df_cleaned = df.copy()
    df_cleaned['tokens'] = df_cleaned['tweets'].apply(lambda tweet: word_tokenize(tweet))
    # remove stop words
    if stop_words:
        stop_words = stopwords.words('english')
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])
    # stemming
    if stemming:
        ps = PorterStemmer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [ps.stem(token) for token in tokens])
    # lemmatization
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(token) for token in tokens])
    return df_cleaned

In [15]:
tokenize_and_preprocess(remove_tags(df_train_pos))

,tweets,tokens
0,i dunno justin read my mention or not . only j...,"[i, dunno, justin, read, my, mention, or, not,..."
1,"because your logic is so dumb , i won't even c...","[because, your, logic, is, so, dumb, ,, i, wo,..."
2,just put casper in a box ! looved the battle ...,"[just, put, casper, in, a, box, !, looved, the..."
3,thanks sir > > don't trip lil mama ... just ke...,"[thanks, sir, >, >, do, n't, trip, lil, mama, ..."
4,visiting my brother tmr is the bestest birthda...,"[visiting, my, brother, tmr, is, the, bestest,..."
...,...,...
97897,hey gina what's up ?,"[hey, gina, what, 's, up, ?]"
97898,"sas 9.1 . 3 and 9.2 , east 5 , s-plus 8 , stat...","[sas, 9.1, ., 3, and, 9.2, ,, east, 5, ,, s-pl..."
97899,um gord ... i just read your profile . i'm not...,"[um, gord, ..., i, just, read, your, profile, ..."
97900,i'm so excited for tomorrow ! look out for two...,"[i, 'm, so, excited, for, tomorrow, !, look, o..."
